In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
# import pyspark.sql.SparkSession <= Error: no module named "pyspark.sql.SparkSession"
# Modules should have short, all-lowercase names. Underscores can be used in the module name if it improves readability. Python packages should also have short, all-lowercase names, although the use of underscores is discouraged.
# Class names should normally use the CapWords convention.
# SparkSession is a class defined in pyspark.sql module
# Where as pyspark.sql.functions and pyspark.sql.types both are standalone modules
# spark = pyspark.sql.SparkSession.builder.appName("Feature Engineering -- UFC Data").getOrCreate()
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Feature Engineering -- UFC Data").getOrCreate()
spark

In [4]:
import os
from os.path import isfile, join


In [5]:
currnet_loc = os.path.abspath("")
data_loc = f"{currnet_loc}/ignoreFiles/"
files = [f for f in os.listdir(data_loc) if isfile(join(data_loc,f))]
files

['data.csv', 'raw_fighter_details.csv', 'raw_total_fight_data.csv']

In [6]:
data = {}
for file in files:
    data[file[:-4]] = spark.read.csv(f'{data_loc}{file}', inferSchema=True, header=True)
    data[file[:-4]].createOrReplaceGlobalTempView(f"data_{file[:-4]}")

In [8]:
print(type(data))
print(data.keys())

<class 'dict'>
dict_keys(['data', 'raw_fighter_details', 'raw_total_fight_data'])


In [12]:
data["data"] = data["data"].where(F.col("date") > "2017-01-01") \
                           .withColumn("Winner_name", F.when(F.col("Winner") == "Red", F.col("R_fighter")).otherwise(F.col("B_fighter"))) \
                           .withColumn("Loser_name", F.when(F.col("Winner") == "Red", F.col("B_fighter")).otherwise(F.col("R_fighter")))

In [13]:
type(data["data"])

pyspark.sql.dataframe.DataFrame

In [15]:
data["data"].limit(1).show()

+------------+-------------+------------+----------+--------------------+------+----------+------------+------------+---------------------+--------------------+------+--------------+-----------------+----------------+-------------------+------------------+---------------------+----------------+-------------------+--------------+-----------------+--------+-------------+----------------+----------+---------+-----------------+--------------------+-----------------+-------------+------------+---------------+------------+-------------------+----------------------+--------------------+--------+------------------+---------------------+--------------------+-----------------------+----------------------+-------------------------+--------------------+-----------------------+------------------+---------------------+------------+-----------------+--------------------+--------------+-------------+---------------------+------------------------+---------------------+-----------------+----------------